In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from bp import data_gather_from_files,run_strategy_optimised,run_strategy_eval
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def error_check(params,n):
    if len(params) != n:
        raise IndexError('The number of parameters is not correct')

def generate_date_ranges_for_walk_forward(start_month_year, end_month_year, day=15, n_months = 1):
    # Initialize an empty list to store the date ranges
    date_ranges = []
    if day not in range(1, 29):
        raise ValueError('Day must be between 1 and 28')
    # Convert the input strings to datetime objects, using the given day
    start_date = datetime.strptime(f"{day} {start_month_year}", '%d %b %Y')
    end_date = datetime.strptime(f"{day} {end_month_year}", '%d %b %Y')
    
    # Generate the date ranges
    current_date = start_date
    while current_date <= end_date:
        next_date = current_date + relativedelta(months=n_months)
        date_range = [current_date.strftime('%d %b %Y'), (next_date - relativedelta(days=1)).strftime('%d %b %Y')]
        date_ranges.append(date_range)
        current_date = next_date
    
    return date_ranges[:-1]

def get_previous_n_months(end_date_str, n_months):
    # Convert the input string to a datetime object
    end_date = datetime.strptime(end_date_str, '%d %b %Y')
    
    # Calculate the start date
    start_date = end_date - relativedelta(months=n_months)
    
    # Create the date range
    date_range = [start_date.strftime('%d %b %Y'), (end_date - relativedelta(days=1)).strftime('%d %b %Y')]
    
    return date_range

In [4]:
def walk_forward_analysis(evaluation_start, evaluation_end, evaluation_day,parameters,optimization_function = None, optimizer_params =[],  lookback_in_months = 6,evaluation_period = 3):
    generated_date_ranges = generate_date_ranges_for_walk_forward(evaluation_start, evaluation_end,evaluation_day,n_months = evaluation_period)
    df = {}
    for dates in generated_date_ranges:
        train_period = get_previous_n_months(dates[0], lookback_in_months)
        train_data = data_gather_from_files(train_period[0],train_period[1])['EURUSD.mid']
        print('Data gathered for training period: ',train_period[0],train_period[1])
        test_data = data_gather_from_files(dates[0],dates[1])['EURUSD.mid']
        print('Data gathered for testing period: ',dates[0],dates[1])
        max_loss, R_PNL,profit,optimal_params = optimization_function(train_data,test_data,parameters,optimizer_params)
        print('Optimal parameters are: ',optimal_params)
        #add optimization phrase evaluation
        a, b, c = run_strategy_optimised(train_data, optimal_params[0],optimal_params[1])
        print('Optimization phrase; Max loss,R_PNL,profit are: ',a, b, c)
        ##
        print('Testing phrase; Max loss, R_PNL,profit are: ',max_loss,R_PNL,profit)
        df[dates[0] +'-'+ dates[1]] = [max_loss, R_PNL,profit]
    df = pd.DataFrame(df).T
    df.columns = ['max_loss','R_PNL','profit']
    return df

In [3]:
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

## Group 1 optimization

In [6]:
import random
from deap import base, creator, tools

def deap_optimiser_g_n(train_data, test_data, parameters, optimization_params):
    ngen = optimization_params[0]  # number of generations
    npop = optimization_params[1]  # number of population

    error_check(parameters,2)
    
    grid_params = parameters[0]
    position_params = parameters[1]

    creator.create("FitnessMax", base.Fitness, weights=(1.0,)) #maximizing
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()

    # Define the genes for our individual
    toolbox.register("G_gene", random.randint, grid_params[0]//grid_params[2], grid_params[1]//grid_params[2])
    toolbox.register("n_gene", random.randint, position_params[0]//position_params[2], position_params[1]//position_params[2])

    # Create an individual with the genes
    toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.G_gene, toolbox.n_gene), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def objective(individual):
        G, n = individual[0]*grid_params[2], individual[1]*position_params[2]
        max_loss, R_PNL, profit = run_strategy_optimised(train_data, G, n)
    
        constraints = [
            max_loss < -500e3
        ]

        if any(constraints):
            return float('-inf'),  # Return large negative value when constraints are not satisfied
        return profit,

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=[grid_params[0]//grid_params[2], position_params[0]//position_params[2]], 
                     up=[grid_params[1]//grid_params[2], position_params[1]//position_params[2]], indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)   ##############
    toolbox.register("evaluate", objective)

    population = toolbox.population(n=npop)
    CXPB, MUTPB = 0.5, 0.2

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    #initiate early stopping
    stagnant_generations = 0  # Counter for generations without improvement
    MAX_STAGNANT_GEN = optimization_params[2]  # Early stopping criterion: stop if no improvement over x generations
    best_fitness_so_far = float('-inf')  # since we're maximizing
    ##
    for gen in range(ngen):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Mutation
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        fitnesses = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit

        #early stopping
        current_best_fitness = max(ind.fitness.values[0] for ind in population)

        if current_best_fitness > best_fitness_so_far:
            best_fitness_so_far = current_best_fitness
            stagnant_generations = 0  # Reset counter
        else:
            stagnant_generations += 1

        if stagnant_generations >= MAX_STAGNANT_GEN:
            print(f"Early stopping on generation {gen} due to no improvement.")
            break
        ##
        population[:] = offspring

    best_ind = tools.selBest(population, 1)[0]
    optimal_g = np.round(best_ind[0]*grid_params[2],5)
    optimal_n = best_ind[1]*position_params[2]
    print("optimisation completed")
    max_loss, R_PNL,profit = run_strategy_optimised(test_data, optimal_g,optimal_n)
    return max_loss, R_PNL,profit,[optimal_g,optimal_n]


## Test 6:1

In [13]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=1)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are:  [0.0025, 1600000]
Optimization phrase; Max loss,R_PNL,profit are:  -256190.08 150142.4643 130942.4644
Testing phrase; Max loss, R_PNL,profit are:  -16000.0 94994.9999 102880.0
Data gathered for training period:  01 Aug 2020 31 Jan 2021
Data gathered for testing period:  01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are:  [0.0035, 2000000]
Optimization phrase; Max loss,R_PNL,profit are:  0 473747.396 479714.0626
Testing phrase; Max loss, R_PNL,profit are:  -63000.0 77000.0 76600.0
Data gathered for training period:  01 Sep 2020 28 Feb 2021
Data gathered for testing period:  01 Mar 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0085, 1900000]
Optimization phrase; Max loss,R_PNL,profit are:  

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,-16000.0,94994.9999,102880.0
01 Feb 2021-28 Feb 2021,-63000.0,77000.0000,76600.0
01 Mar 2021-31 Mar 2021,-134520.0,5383.3333,-134520.0
01 Apr 2021-30 Apr 2021,-48750.0,11700.0000,-48750.0
01 May 2021-31 May 2021,0.0,32500.0000,32200.0
01 Jun 2021-30 Jun 2021,-44000.0,16000.0000,-44000.0
01 Jul 2021-31 Jul 2021,0.0,20000.0000,20000.0
01 Aug 2021-31 Aug 2021,-45000.0,4400.0000,8800.0
01 Sep 2021-30 Sep 2021,-10200.0,20400.0000,-2720.0
01 Oct 2021-31 Oct 2021,0.0,8320.0000,8320.0


In [1]:
102880+76600-134520-48750+32200-44000+20000+8800-2720+8320-38870+130000

109940

## Test 12:1

In [7]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=12,evaluation_period=1)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Jan 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are:  [0.0085, 1700000]
Optimization phrase; Max loss,R_PNL,profit are:  -72250.0 514016.6666 503986.6666
Testing phrase; Max loss, R_PNL,profit are:  0 32512.5 39780.0
Data gathered for training period:  01 Feb 2020 31 Jan 2021
Data gathered for testing period:  01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are:  [0.009, 1800000]
Optimization phrase; Max loss,R_PNL,profit are:  -97200.0 486712.5 486712.5
Testing phrase; Max loss, R_PNL,profit are:  0 78660.0 78660.0
Data gathered for training period:  01 Mar 2020 28 Feb 2021
Data gathered for testing period:  01 Mar 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.006, 1300000]
Optimization phrase; Max loss,R_PNL,profit are:  -93600.0 548314.3501

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,0.0,32512.5,39780.0
01 Feb 2021-28 Feb 2021,0.0,78660.0,78660.0
01 Mar 2021-31 Mar 2021,-70850.0,18200.0,-70850.0
01 Apr 2021-30 Apr 2021,-180000.0,-37750.0,-180000.0
01 May 2021-31 May 2021,-9000.0,47700.0,48600.0
01 Jun 2021-30 Jun 2021,-150000.0,-137650.0,-137650.0
01 Jul 2021-31 Jul 2021,-13300.0,39900.0,39900.0
01 Aug 2021-31 Aug 2021,0.0,21060.0,21060.0
01 Sep 2021-30 Sep 2021,-13000.0,26000.0,-200.0
01 Oct 2021-31 Oct 2021,-15200.0,20320.0,20320.0


## Test 6:3

In [9]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0025, 2000000]
Optimization phrase; Max loss,R_PNL,profit are:  -264108.8869 148947.5583 127750.6832
Training phrase; Max loss, R_PNL,profit are:  -41600.0 62500.0001 -41600.0
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are:  [0.0095, 1600000]
Optimization phrase; Max loss,R_PNL,profit are:  -76000.0 258400.0 258400.0
Training phrase; Max loss, R_PNL,profit are:  -121600.0 77320.0 98400.0
Data gathered for training period:  01 Jan 2021 30 Jun 2021
Data gathered for testing period:  01 Jul 2021 30 Sep 2021
optimisation completed
Optimal parameters are:  [0.0095, 2000000]
Optimization phrase; Max loss,R_PNL,profit

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-41600.0,62500.0001,-41600.0
01 Apr 2021-30 Jun 2021,-121600.0,77320.0000,98400.0
01 Jul 2021-30 Sep 2021,0.0,68900.0000,61800.0
01 Oct 2021-31 Dec 2021,-28800.0,48888.8889,66240.0


## Group 1 Test run again for stability (Tune parameters)

In [10]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0025, 2000000]
Optimization phrase; Max loss,R_PNL,profit are:  -264108.8869 148947.5583 127750.6832
Training phrase; Max loss, R_PNL,profit are:  -41600.0 62500.0001 -41600.0
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are:  [0.009, 1300000]
Optimization phrase; Max loss,R_PNL,profit are:  -81900.0 245699.9999 245700.0
Training phrase; Max loss, R_PNL,profit are:  -93600.0 72306.0 91000.0
Data gathered for training period:  01 Jan 2021 30 Jun 2021
Data gathered for testing period:  01 Jul 2021 30 Sep 2021
optimisation completed
Optimal parameters are:  [0.0095, 2000000]
Optimization phrase; Max loss,R_PNL,profi

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-41600.0,62500.0001,-41600.0
01 Apr 2021-30 Jun 2021,-93600.0,72306.0000,91000.0
01 Jul 2021-30 Sep 2021,0.0,68900.0000,61800.0
01 Oct 2021-31 Dec 2021,-28800.0,48888.8889,66240.0


In [3]:
# compare to optuna
data = data_gather_from_files('1 oct 2020','31 mar 2021')['EURUSD.mid']
run_strategy_optimised(data, 0.0095,1600000)


(-76000.0, 258400.0, 258400.0)

In [24]:
-41600.0+91000.0+61800.0+66240.0

177440.0

### parameters seem stable with n_trials = 10 , npop = 100, no early stopping, run time = 13 mins

In [19]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0025, 1600000]
Optimization phrase; Max loss,R_PNL,profit are:  -256190.08 150142.4643 130942.4644
Testing phrase; Max loss, R_PNL,profit are:  -21600.0 91199.9997 -21600.0
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are:  [0.009, 1300000]
Optimization phrase; Max loss,R_PNL,profit are:  -81900.0 245699.9999 245700.0
Testing phrase; Max loss, R_PNL,profit are:  -93600.0 72306.0 91000.0
Data gathered for training period:  01 Jan 2021 30 Jun 2021
Data gathered for testing period:  01 Jul 2021 30 Sep 2021
optimisation completed
Optimal parameters are:  [0.0095, 1900000]
Optimization phrase; Max loss,R_PNL,profit ar

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-21600.0,91199.9997,-21600.0
01 Apr 2021-30 Jun 2021,-93600.0,72306.0000,91000.0
01 Jul 2021-30 Sep 2021,0.0,65455.0000,58710.0
01 Oct 2021-31 Dec 2021,-28800.0,48888.8889,66240.0


In [23]:
-21600.0+91000.0+58710.0+66240.0

194350.0

### 2 yr test

In [21]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2020','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Jul 2019 31 Dec 2019
Data gathered for testing period:  01 Jan 2020 31 Mar 2020
optimisation completed
Optimal parameters are:  [0.0055, 1100000]
Optimization phrase; Max loss,R_PNL,profit are:  -78650.0 139111.5581 171270.0
Testing phrase; Max loss, R_PNL,profit are:  -124935.7144 -1723.9287 -56515.7144
Data gathered for training period:  01 Oct 2019 31 Mar 2020
Data gathered for testing period:  01 Apr 2020 30 Jun 2020
optimisation completed
Optimal parameters are:  [0.008, 1300000]
Optimization phrase; Max loss,R_PNL,profit are:  -62400.0 292630.0 292630.0
Testing phrase; Max loss, R_PNL,profit are:  -10400.0 87620.0 89960.0
Data gathered for training period:  01 Jan 2020 30 Jun 2020
Data gathered for testing period:  01 Jul 2020 30 Sep 2020
optimisation completed
Optimal parameters are:  [0.0085, 1800000]
Optimization phrase; Max loss,R_PNL,profit are

,max_loss,R_PNL,profit
01 Jan 2020-31 Mar 2020,-124935.7144,-1723.9287,-56515.7144
01 Apr 2020-30 Jun 2020,-10400.0000,87620.0000,89960.0000
01 Jul 2020-30 Sep 2020,-150000.0000,48900.0000,48900.0000
01 Oct 2020-31 Dec 2020,-18562.5000,-362.5000,-6162.5000
01 Jan 2021-31 Mar 2021,-21600.0000,91199.9997,-21600.0000
01 Apr 2021-30 Jun 2021,-117000.0000,28697.2788,57500.0000
01 Jul 2021-30 Sep 2021,0.0000,68900.0000,61800.0000
01 Oct 2021-31 Dec 2021,-28800.0000,48888.8889,66240.0000


In [22]:
-56515.7144+89960.0000+48900.0000-6162.5000-21600.0000+57500.0000+61800.0000+66240.0000

240121.7856

### Train:Test = 3:1 try

In [6]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])

n_combination = n_grid_params*n_lot_params
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('total_number_of_combinations:-',(n_combination))

# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params]
results = walk_forward_analysis('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_g_n,optimizer_params=optimizer_param,lookback_in_months=3,evaluation_period=1)
results

number of grid params:- 18.0
number of lot params:- 19.0
total_number_of_combinations:- 342.0
Data gathered for training period:  01 Oct 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are:  [0.0015, 1700000]
Optimization phrase; Max loss,R_PNL,profit are:  -30600.0 129806.3098 123080.0
Testing phrase; Max loss, R_PNL,profit are:  -33150.0 89419.9997 89420.0
Data gathered for training period:  01 Nov 2020 31 Jan 2021
Data gathered for testing period:  01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are:  [0.0015, 1700000]
Optimization phrase; Max loss,R_PNL,profit are:  -76500.0 201619.9997 201620.0
Testing phrase; Max loss, R_PNL,profit are:  -107100.0 -80324.9999 -105910.0
Data gathered for training period:  01 Dec 2020 28 Feb 2021
Data gathered for testing period:  01 Mar 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0085, 2000000]
Optimization phrase; Max loss,R_PNL,profit 

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,-33150.0,89419.9997,89420.0
01 Feb 2021-28 Feb 2021,-107100.0,-80324.9999,-105910.0
01 Mar 2021-31 Mar 2021,-141600.0,5666.6667,-141600.0
01 Apr 2021-30 Apr 2021,-3700.0,950.0000,-3700.0
01 May 2021-31 May 2021,-34000.0,11333.3333,3400.0
01 Jun 2021-30 Jun 2021,-72000.0,-2400.0000,-48000.0
01 Jul 2021-31 Jul 2021,0.0,30400.0000,30400.0
01 Aug 2021-31 Aug 2021,0.0,31400.0000,31400.0
01 Sep 2021-30 Sep 2021,-58800.0,-13745.5998,-47600.0
01 Oct 2021-31 Oct 2021,-28000.0,71812.5004,57200.0


In [18]:
# Given profit data
profits = [
    89420.0,
    -105910.0,
    -141600.0,
    -3700.0,
    3400.0,
    -48000.0,
    30400.0,
    31400.0,
    -47600.0,
    57200.0,
    14200.0,
    130000.0
]

sum(profits)

9210.0

## Group 2 Optimization

In [4]:
import random
from deap import base, creator, tools

def deap_optimiser_multiplier(train_data, test_data, parameters, optimization_params):
    ngen = optimization_params[0]  # number of generations
    npop = optimization_params[1]  # number of population

    error_check(parameters,4)
    
    grid_params = parameters[0]
    position_params = parameters[1]
    multiplier_params = parameters[2]
    lookback_params = parameters[3]

    creator.create("FitnessMax", base.Fitness, weights=(1.0,)) #maximizing
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()

    # Define the genes for our individual
    toolbox.register("G_gene", random.randint, grid_params[0]//grid_params[2], grid_params[1]//grid_params[2])
    toolbox.register("n_gene", random.randint, position_params[0]//position_params[2], position_params[1]//position_params[2])

    toolbox.register("multiplier_gene", random.randint, multiplier_params[0]//multiplier_params[2], multiplier_params[1]//multiplier_params[2])
    toolbox.register("lookback_gene", random.choice, lookback_params)

    CXPB, MUTPB = 0.5, 0.2

    def custom_mutate(individual): 
        if random.random() < MUTPB:
            individual[0] = random.randint(grid_params[0]//grid_params[2], grid_params[1]//grid_params[2])
        if random.random() < MUTPB:
            individual[1] = random.randint(position_params[0]//position_params[2], position_params[1]//position_params[2])
        if random.random() < MUTPB:
            individual[2] = random.randint(multiplier_params[0]//multiplier_params[2], multiplier_params[1]//multiplier_params[2])
        if random.random() < MUTPB:
            individual[3] = random.choice(lookback_params)
        return individual,

    toolbox.register("mutate", custom_mutate)

    # Create an individual with the genes
    toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.G_gene, toolbox.n_gene, toolbox.multiplier_gene, toolbox.lookback_gene), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def objective(individual):
        G, n, multiplier, lookback = individual[0]*grid_params[2], individual[1]*position_params[2], individual[2]*multiplier_params[2], individual[3]

        max_loss, R_PNL, profit = run_strategy_optimised(train_data, G, n, multiplier = multiplier,lookback = lookback)
    
        constraints = [
            max_loss < -500e3
        ]

        if any(constraints):
            return float('-inf'),  # Return large negative value when constraints are not satisfied
        return profit,

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("select", tools.selTournament, tournsize=3)   
    toolbox.register("evaluate", objective)

    population = toolbox.population(n=npop)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    #initiate early stopping
    stagnant_generations = 0  # Counter for generations without improvement
    MAX_STAGNANT_GEN = optimization_params[2]  # Early stopping criterion: stop if no improvement over x generations
    best_fitness_so_far = float('-inf')  # since we're maximizing
    ##
    for gen in range(ngen):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Mutation
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        fitnesses = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit

        #early stopping
        current_best_fitness = max(ind.fitness.values[0] for ind in population)

        if current_best_fitness > best_fitness_so_far:
            best_fitness_so_far = current_best_fitness
            stagnant_generations = 0  # Reset counter
        else:
            stagnant_generations += 1

        if stagnant_generations >= MAX_STAGNANT_GEN:
            print(f"Early stopping on generation {gen} due to no improvement.")
            break
        ##
        population[:] = offspring

    best_ind = tools.selBest(population, 1)[0]
    optimal_g = np.round(best_ind[0]*grid_params[2],5)
    optimal_n = best_ind[1]*position_params[2]
    optimal_m = np.round(best_ind[2] *multiplier_params[2],5)
    optimal_l = best_ind[3]

    print("optimisation completed")
    max_loss, R_PNL,profit = run_strategy_optimised(test_data, optimal_g,optimal_n,multiplier = optimal_m,lookback = optimal_l)
    return max_loss, R_PNL,profit,[optimal_g,optimal_n,optimal_m,optimal_l]


In [5]:
def walk_forward_analysis_multiplier(evaluation_start, evaluation_end, evaluation_day,parameters,optimization_function = None, optimizer_params =[],  lookback_in_months = 6,evaluation_period = 3):
    generated_date_ranges = generate_date_ranges_for_walk_forward(evaluation_start, evaluation_end,evaluation_day,n_months = evaluation_period)
    df = {}
    for dates in generated_date_ranges:
        train_period = get_previous_n_months(dates[0], lookback_in_months)
        train_data = data_gather_from_files(train_period[0],train_period[1])['EURUSD.mid']
        print('Data gathered for training period: ',train_period[0],train_period[1])
        test_data = data_gather_from_files(dates[0],dates[1])['EURUSD.mid']
        print('Data gathered for testing period: ',dates[0],dates[1])
        max_loss, R_PNL,profit,optimal_params = optimization_function(train_data,test_data,parameters,optimizer_params)
        print('Optimal parameters are: ',optimal_params)
        #add optimization phrase evaluation
        a, b, c = run_strategy_optimised(train_data, optimal_params[0],optimal_params[1], multiplier = optimal_params[2], lookback=optimal_params[3])
        print('Optimization phrase; Max loss,R_PNL,profit are: ',a, b, c)
        ##
        print('Testing phrase; Max loss, R_PNL,profit are: ',max_loss,R_PNL,profit)
        df[dates[0] +'-'+ dates[1]] = [max_loss, R_PNL,profit]
    df = pd.DataFrame(df).T
    df.columns = ['max_loss','R_PNL','profit']
    return df

In [36]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 2 #NGEN
npop = 5
early_stopping_gen = 10 # early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]
results = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results

number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0025, 1800000, 2.3, 200]
Optimization phrase; Max loss,R_PNL,profit are:  -53955.44 242771.7571 248171.5631
Testing phrase; Max loss, R_PNL,profit are:  -13500.0 144091.4503 110979.909
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are:  [0.0065, 1500000, 2.6, 9]
Optimization phrase; Max loss,R_PNL,profit are:  0 418470.0 409602.0
Testing phrase; Max loss, R_PNL,profit are:  -214687.5 -204937.5 -190537.5
Data gathered for training period:  01 Jan 2021 30 Jun 2021
Data gathered for testing period:  01 Jul 2021 30 Sep 2021
optimisation completed
Optim

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-13500.0,144091.4503,110979.909
01 Apr 2021-30 Jun 2021,-214687.5,-204937.5000,-190537.500
01 Jul 2021-30 Sep 2021,0.0,48230.0000,43260.000
01 Oct 2021-31 Dec 2021,-25200.0,92653.0464,96452.000


In [31]:
data = data_gather_from_files('1 jul 2020','31 dec 2020')['EURUSD.mid']
run_strategy_optimised(data, 0.004, 2000000, multiplier=1.9, lookback=100)

(-112880.0, 154152.0, 150466.0)

In [33]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 10 #NGEN
npop = 100
early_stopping_gen = 10 # early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]
results = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results

number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are:  [0.0055, 1000000, 2.4, 100]
Optimization phrase; Max loss,R_PNL,profit are:  0 781521.1721 877340.0
Testing phrase; Max loss, R_PNL,profit are:  -16720.0 81699.9999 81700.0
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are:  [0.0055, 1200000, 2.6, 20]
Optimization phrase; Max loss,R_PNL,profit are:  -3960.0 529560.0001 540816.0
Testing phrase; Max loss, R_PNL,profit are:  -242152.9412 -235552.9412 -228832.9412
Data gathered for training period:  01 Jan 2021 30 Jun 2021
Data gathered for testing period:  01 Jul 2021 30 Sep 2021
optimisation completed
O

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-16720.0000,81699.9999,81700.0000
01 Apr 2021-30 Jun 2021,-242152.9412,-235552.9412,-228832.9412
01 Jul 2021-30 Sep 2021,-11466.0000,78032.8421,12885.6000
01 Oct 2021-31 Dec 2021,-22625.0000,174595.0857,158225.0000


## Revised walk forward to return both table

In [5]:
def walk_forward_analysis_multiplier(evaluation_start, evaluation_end, evaluation_day, parameters, optimization_function=None, optimizer_params=[], lookback_in_months=6, evaluation_period=3):
    generated_date_ranges = generate_date_ranges_for_walk_forward(evaluation_start, evaluation_end, evaluation_day, n_months=evaluation_period)
    #replace df here
    testing_results = {} # use to be df
    optimization_results = {}
    
    for dates in generated_date_ranges:
        train_period = get_previous_n_months(dates[0], lookback_in_months)
        train_data = data_gather_from_files(train_period[0], train_period[1])['EURUSD.mid']
        print('Data gathered for training period:', train_period[0], train_period[1])
        test_data = data_gather_from_files(dates[0], dates[1])['EURUSD.mid']
        print('Data gathered for testing period:', dates[0], dates[1])
        max_loss, R_PNL, profit, optimal_params = optimization_function(train_data, test_data, parameters, optimizer_params)
        print('Optimal parameters are:', optimal_params)
        
        # Optimization phase evaluation
        opt_max_loss, opt_R_PNL, opt_profit = run_strategy_optimised(train_data, optimal_params[0], optimal_params[1], multiplier=optimal_params[2], lookback=optimal_params[3])
        print('Optimization phase; Max loss, R_PNL, profit are:', opt_max_loss, opt_R_PNL, opt_profit)
        optimization_results[train_period[0] + '-' + train_period[1]] = [opt_max_loss, opt_R_PNL, opt_profit]
        
        # Testing phase results
        print('Testing phase; Max loss, R_PNL, profit are:', max_loss, R_PNL, profit)
        testing_results[dates[0] + '-' + dates[1]] = [max_loss, R_PNL, profit]

    testing_df = pd.DataFrame(testing_results).T
    testing_df.columns = ['max_loss', 'R_PNL', 'profit']

    optimization_df = pd.DataFrame(optimization_results).T
    optimization_df.columns = ['max_loss', 'R_PNL', 'profit']
    
    return testing_df, optimization_df


In [74]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jul 2020 31 Dec 2020
Data gathered for testing period: 01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are: [0.006, 1400000, 2.5, 30]
Optimization phase; Max loss, R_PNL, profit are: 0 768599.2146 866740.0
Testing phase; Max loss, R_PNL, profit are: -77420.0 2645.4545 -77420.0
Data gathered for training period: 01 Oct 2020 31 Mar 2021
Data gathered for testing period: 01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are: [0.0025, 100000, 2.5, 20]
Optimization phase; Max loss, R_PNL, profit are: -12523.4375 474642.0129 565822.8125
Testing phase; Max loss, R_PNL, profit are: -21056.7436 5122.1721 32240.1314
Data gathered for training period: 01 Jan 2021 30 Jun 2021
Data gathered for testing period: 01 Jul 2021 30 Sep 2021
optimisation completed
Optimal param

In [75]:
optimization_df

,max_loss,R_PNL,profit
01 Jul 2020-31 Dec 2020,0.0000,768599.2146,866740.0000
01 Oct 2020-31 Mar 2021,-12523.4375,474642.0129,565822.8125
01 Jan 2021-30 Jun 2021,-4500.0000,457075.1963,465515.0000
01 Apr 2021-30 Sep 2021,-1760.0000,382122.3998,382122.4000


In [76]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-77420.0000,2645.4545,-77420.0000
01 Apr 2021-30 Jun 2021,-21056.7436,5122.1721,32240.1314
01 Jul 2021-30 Sep 2021,-16245.0000,71701.3923,9617.0000
01 Oct 2021-31 Dec 2021,-49984.0000,166320.0000,177091.2000


In [81]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jul 2020 31 Dec 2020
Data gathered for testing period: 01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are: [0.006, 1100000, 2.4, 200]
Optimization phase; Max loss, R_PNL, profit are: 0 712270.7209 826465.2
Testing phase; Max loss, R_PNL, profit are: -154000.88 -4045.358 -154000.88
Data gathered for training period: 01 Oct 2020 31 Mar 2021
Data gathered for testing period: 01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are: [0.0035, 1500000, 2.5, 20]
Optimization phase; Max loss, R_PNL, profit are: -34125.0 578812.5002 575737.5
Testing phase; Max loss, R_PNL, profit are: -236999.9999 -214249.9999 -205499.9999
Data gathered for training period: 01 Jan 2021 30 Jun 2021
Data gathered for testing period: 01 Jul 2021 30 Sep 2021
optimisation completed
Optimal

In [82]:
optimization_df

,max_loss,R_PNL,profit
01 Jul 2020-31 Dec 2020,0.0,712270.7209,826465.200
01 Oct 2020-31 Mar 2021,-34125.0,578812.5002,575737.500
01 Jan 2021-30 Jun 2021,-20244.0,494476.9350,495796.000
01 Apr 2021-30 Sep 2021,-504.0,383209.5228,458876.428


In [83]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-154000.8800,-4045.3580,-154000.8800
01 Apr 2021-30 Jun 2021,-236999.9999,-214249.9999,-205499.9999
01 Jul 2021-30 Sep 2021,-212940.0000,-110224.7995,-110224.8000
01 Oct 2021-31 Dec 2021,-26883.8400,156404.3831,156404.3822


### experiment

In [77]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 30 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jul 2020 31 Dec 2020
Data gathered for testing period: 01 Jan 2021 31 Mar 2021
optimisation completed
Optimal parameters are: [0.006, 1100000, 2.4, 100]
Optimization phase; Max loss, R_PNL, profit are: 0 712270.7209 826465.2
Testing phase; Max loss, R_PNL, profit are: -154000.88 -4045.358 -154000.88
Data gathered for training period: 01 Oct 2020 31 Mar 2021
Data gathered for testing period: 01 Apr 2021 30 Jun 2021
optimisation completed
Optimal parameters are: [0.0035, 1400000, 2.6, 20]
Optimization phase; Max loss, R_PNL, profit are: -40180.0 599760.0 599592.0
Testing phase; Max loss, R_PNL, profit are: -242550.7409 -220745.7409 -211190.7409
Data gathered for training period: 01 Jan 2021 30 Jun 2021
Data gathered for testing period: 01 Jul 2021 30 Sep 2021
optimisation completed
Optimal pa

In [78]:
optimization_df

,max_loss,R_PNL,profit
01 Jul 2020-31 Dec 2020,0.0,712270.7209,826465.200
01 Oct 2020-31 Mar 2021,-40180.0,599760.0000,599592.000
01 Jan 2021-30 Jun 2021,-8250.0,423036.3984,440000.000
01 Apr 2021-30 Sep 2021,-11100.0,330944.8041,443835.504


In [79]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2021-31 Mar 2021,-154000.8800,-4045.3580,-154000.8800
01 Apr 2021-30 Jun 2021,-242550.7409,-220745.7409,-211190.7409
01 Jul 2021-30 Sep 2021,-79062.5000,-357.5000,-357.5000
01 Oct 2021-31 Dec 2021,-190445.9999,-110541.9785,-131204.3999


### Train:Test 3:1 ratio

In [84]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=3,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Oct 2020 31 Dec 2020
Data gathered for testing period: 01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are: [0.0065, 500000, 2.9, 150]
Optimization phase; Max loss, R_PNL, profit are: 0 339269.3955 565075.75
Testing phase; Max loss, R_PNL, profit are: -20540.0 7537.0 7537.0
Data gathered for training period: 01 Nov 2020 31 Jan 2021
Data gathered for testing period: 01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are: [0.0015, 500000, 2.2, 200]
Optimization phase; Max loss, R_PNL, profit are: -19176.0 515132.7983 505508.0
Testing phase; Max loss, R_PNL, profit are: -118506.0 -84146.2304 -51964.64
Data gathered for training period: 01 Dec 2020 28 Feb 2021
Data gathered for testing period: 01 Mar 2021 31 Mar 2021
optimisation completed
Optimal parameters are

In [85]:
optimization_df

,max_loss,R_PNL,profit
01 Oct 2020-31 Dec 2020,0.0,339269.3955,565075.7500
01 Nov 2020-31 Jan 2021,-19176.0,515132.7983,505508.0000
01 Dec 2020-28 Feb 2021,-114000.0,320921.8750,320381.5625
01 Jan 2021-31 Mar 2021,-400.0,123709.9241,384144.4270
01 Feb 2021-30 Apr 2021,-26325.0,345281.9402,351266.5000
01 Mar 2021-31 May 2021,-19200.0,292679.4610,291104.0084
01 Apr 2021-30 Jun 2021,-22819.5,290369.3645,301570.5000
01 May 2021-31 Jul 2021,-38400.0,239660.3202,291900.0000
01 Jun 2021-31 Aug 2021,-19890.0,299578.5001,294582.6000
01 Jul 2021-30 Sep 2021,0.0,153600.0000,256588.8000


In [86]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,-20540.000,7537.0000,7537.000
01 Feb 2021-28 Feb 2021,-118506.000,-84146.2304,-51964.640
01 Mar 2021-31 Mar 2021,-138474.375,-50646.8750,-138474.375
01 Apr 2021-30 Apr 2021,-10780.000,-5345.1474,-10780.000
01 May 2021-31 May 2021,-90480.000,-76348.3726,-85636.200
01 Jun 2021-30 Jun 2021,-16639.992,141695.9360,159743.894
01 Jul 2021-31 Jul 2021,-87087.000,-67529.0002,-65824.000
01 Aug 2021-31 Aug 2021,-14400.000,5360.0000,5360.000
01 Sep 2021-30 Sep 2021,-79601.600,11700.0000,-79601.600
01 Oct 2021-31 Oct 2021,0.000,6720.0000,6720.000


In [17]:
7537-51964.640-138474.375-10780.000-85636.200+159743.894-65824+5360-79601.600+6720-24310+92684.52

-184545.401

## Test 12:1

In [10]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=12,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jan 2020 31 Dec 2020
Data gathered for testing period: 01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are: [0.003, 1100000, 2.9, 30]
Optimization phase; Max loss, R_PNL, profit are: -12540.0 1617208.0466 1657205.0
Testing phase; Max loss, R_PNL, profit are: -56100.0 44192.3955 31103.6
Data gathered for training period: 01 Feb 2020 31 Jan 2021
Data gathered for testing period: 01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are: [0.0045, 1500000, 2.1, 50]
Optimization phase; Max loss, R_PNL, profit are: -138780.0 1507966.1264 1494568.35
Testing phase; Max loss, R_PNL, profit are: -41310.0 45900.0 25770.0
Data gathered for training period: 01 Mar 2020 28 Feb 2021
Data gathered for testing period: 01 Mar 2021 31 Mar 2021
optimisation completed
Optimal param

In [11]:
optimization_df

,max_loss,R_PNL,profit
01 Jan 2020-31 Dec 2020,-12540.000,1.617208e+06,1.657205e+06
01 Feb 2020-31 Jan 2021,-138780.000,1.507966e+06,1.494568e+06
01 Mar 2020-28 Feb 2021,-28452.000,1.498928e+06,1.436260e+06
01 Apr 2020-31 Mar 2021,-51656.000,1.232663e+06,1.338489e+06
01 May 2020-30 Apr 2021,-27216.500,1.305081e+06,1.438105e+06
01 Jun 2020-31 May 2021,-36000.000,1.074960e+06,1.033920e+06
01 Jul 2020-30 Jun 2021,-31920.000,1.005893e+06,1.016130e+06
01 Aug 2020-31 Jul 2021,-5760.000,7.416285e+05,7.735665e+05
01 Sep 2020-31 Aug 2021,0.000,1.131840e+06,1.109160e+06
01 Oct 2020-30 Sep 2021,0.000,6.213513e+05,6.213513e+05


In [12]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,-56100.000,44192.3955,31103.6000
01 Feb 2021-28 Feb 2021,-41310.000,45900.0000,25770.0000
01 Mar 2021-31 Mar 2021,-12480.000,112977.0929,79327.2000
01 Apr 2021-30 Apr 2021,-7600.000,16660.0376,67280.0000
01 May 2021-31 May 2021,-23826.500,25754.4722,34498.1000
01 Jun 2021-30 Jun 2021,-108000.000,0.0000,-108000.0000
01 Jul 2021-31 Jul 2021,0.000,60949.9333,44130.0000
01 Aug 2021-31 Aug 2021,-31092.000,22530.9271,37548.0000
01 Sep 2021-30 Sep 2021,-5400.000,83700.0000,83220.0000
01 Oct 2021-31 Oct 2021,0.000,6300.0000,6300.0000


# Summary
Data use : Year 2021

Group 1: 342 combs

Test with walkforward 6-3
* ngen=10 npop=100, 13 mins >> good and stable result

Test with walkforward 3-1
* ngen=10 npop=100,  21 mins >>  walkforward>> profit = 9210

Test with walkforward 12-1
* ngen=10 npop=100,  76 mins >>  walkforward not so good

Group 2 : 47880 combs 

Test with walkforward 6-3

* ngen= 50 npop = 100, 56 mins >> walk forward can be good and bad

* ngen= 30 npop = 100, 34 mins >> opt okay but walk forward really bad

Test with walkforward 3-1

* ngen= 50 npop = 100 >> walk forward >> Loss

Test with walkforward 12-1

* ngen= 50 npop = 100,  6 hrs >> walk forward good

In [6]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2021','jan 2022',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jul 2020 31 Dec 2020
Data gathered for testing period: 01 Jan 2021 31 Jan 2021
optimisation completed
Optimal parameters are: [0.006, 700000, 2.8, 100]
Optimization phase; Max loss, R_PNL, profit are: 0 625934.1421 842199.62
Testing phase; Max loss, R_PNL, profit are: -31079.982 5880.012 18720.7996
Data gathered for training period: 01 Aug 2020 31 Jan 2021
Data gathered for testing period: 01 Feb 2021 28 Feb 2021
optimisation completed
Optimal parameters are: [0.0065, 1900000, 2.1, 50]
Optimization phase; Max loss, R_PNL, profit are: -12350.0 652545.9502 656676.1
Testing phase; Max loss, R_PNL, profit are: -27170.0 101393.5 78707.5
Data gathered for training period: 01 Sep 2020 28 Feb 2021
Data gathered for testing period: 01 Mar 2021 31 Mar 2021
optimisation completed
Optimal parameters ar

In [7]:
optimization_df

,max_loss,R_PNL,profit
01 Jul 2020-31 Dec 2020,0.0,625934.1421,842199.620
01 Aug 2020-31 Jan 2021,-12350.0,652545.9502,656676.100
01 Sep 2020-28 Feb 2021,-26000.0,574979.1666,535800.000
01 Oct 2020-31 Mar 2021,-104220.0,441890.0848,578211.456
01 Nov 2020-30 Apr 2021,-39232.8,701709.8894,800185.600
01 Dec 2020-31 May 2021,-58860.0,627156.0004,627156.000
01 Jan 2021-30 Jun 2021,-16848.0,495008.8869,503232.000
01 Feb 2021-31 Jul 2021,-54000.0,359500.0000,350000.000
01 Mar 2021-31 Aug 2021,-24000.0,492667.1266,480488.000
01 Apr 2021-30 Sep 2021,-1360.0,394971.1997,394971.200


In [8]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2021-31 Jan 2021,-31079.982,5880.0120,18720.7996
01 Feb 2021-28 Feb 2021,-27170.000,101393.5000,78707.5000
01 Mar 2021-31 Mar 2021,-137600.000,-21666.6667,-137600.0000
01 Apr 2021-30 Apr 2021,-7200.000,25635.1370,97760.0000
01 May 2021-31 May 2021,-19507.200,16951.8995,16027.2000
01 Jun 2021-30 Jun 2021,0.000,82651.3422,83052.0000
01 Jul 2021-31 Jul 2021,-84096.000,-65351.9997,-63432.0000
01 Aug 2021-31 Aug 2021,-18000.000,18000.0000,13800.0000
01 Sep 2021-30 Sep 2021,-64000.000,-27120.0000,-62400.0000
01 Oct 2021-31 Oct 2021,-43792.000,14069.4194,-3998.4000


In [16]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2014','jan 2023',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jul 2013 31 Dec 2013
Data gathered for testing period: 01 Jan 2014 31 Jan 2014
optimisation completed
Optimal parameters are: [0.0055, 300000, 2.6, 150]
Optimization phase; Max loss, R_PNL, profit are: -7524.0 545693.9641 740260.224
Testing phase; Max loss, R_PNL, profit are: -20248.8 13092.9645 -21.36
Data gathered for training period: 01 Aug 2013 31 Jan 2014
Data gathered for testing period: 01 Feb 2014 28 Feb 2014
optimisation completed
Optimal parameters are: [0.0095, 1700000, 1.3, 20]
Optimization phase; Max loss, R_PNL, profit are: -16150.0 430228.9025 436702.8691
Testing phase; Max loss, R_PNL, profit are: -48450.0 0.0 -39389.0
Data gathered for training period: 01 Sep 2013 28 Feb 2014
Data gathered for testing period: 01 Mar 2014 31 Mar 2014
optimisation completed
Optimal parameters

In [17]:
optimization_df

,max_loss,R_PNL,profit
01 Jul 2013-31 Dec 2013,-7524.00,5.456940e+05,7.402602e+05
01 Aug 2013-31 Jan 2014,-16150.00,4.302289e+05,4.367029e+05
01 Sep 2013-28 Feb 2014,-8400.00,5.370153e+05,5.225918e+05
01 Oct 2013-31 Mar 2014,0.00,4.036980e+05,4.215866e+05
01 Nov 2013-30 Apr 2014,-32400.00,5.058000e+05,5.176800e+05
...,...,...,...
01 Feb 2022-31 Jul 2022,-18360.00,8.348534e+05,9.700901e+05
01 Mar 2022-31 Aug 2022,-84000.00,1.118227e+06,1.117681e+06
01 Apr 2022-30 Sep 2022,-46800.00,9.726738e+05,8.775900e+05
01 May 2022-31 Oct 2022,-70080.00,7.872000e+05,8.361600e+05


In [18]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2014-31 Jan 2014,-20248.800,13092.9645,-21.3600
01 Feb 2014-28 Feb 2014,-48450.000,0.0000,-39389.0000
01 Mar 2014-31 Mar 2014,-21839.988,11606.0167,11606.0167
01 Apr 2014-30 Apr 2014,-140.000,23351.0136,29568.0000
01 May 2014-31 May 2014,-117600.000,-117600.0000,-117600.0000
...,...,...,...
01 Aug 2022-31 Aug 2022,0.000,121590.8432,298933.9200
01 Sep 2022-30 Sep 2022,-112000.000,17045.0000,17045.0000
01 Oct 2022-31 Oct 2022,-117000.000,-33900.0000,-32400.0000
01 Nov 2022-30 Nov 2022,-152400.000,-92400.0000,-90672.0000


In [22]:
df = pd.DataFrame(optimization_df)

# Write the DataFrame to a CSV file
df.to_csv('output_train_6_1_group2_.csv', index=True)

In [21]:
df = pd.DataFrame(testing_df)

# Write the DataFrame to a CSV file
df.to_csv('output_test_6_1_group2_.csv', index=True)

In [23]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2014','jan 2015',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=12,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jan 2013 31 Dec 2013
Data gathered for testing period: 01 Jan 2014 31 Jan 2014
optimisation completed
Optimal parameters are: [0.0045, 2000000, 2.2, 50]
Optimization phase; Max loss, R_PNL, profit are: -95400.0 806919.9998 806920.0
Testing phase; Max loss, R_PNL, profit are: -51120.0 -23080.0 -42176.0
Data gathered for training period: 01 Feb 2013 31 Jan 2014
Data gathered for testing period: 01 Feb 2014 28 Feb 2014
optimisation completed
Optimal parameters are: [0.0045, 2000000, 2.5, 30]
Optimization phase; Max loss, R_PNL, profit are: -4500.0 860200.0 860200.0
Testing phase; Max loss, R_PNL, profit are: -9000.0 117000.0 115100.0
Data gathered for training period: 01 Mar 2013 28 Feb 2014
Data gathered for testing period: 01 Mar 2014 31 Mar 2014
optimisation completed
Optimal parameters are

In [24]:
optimization_df

,max_loss,R_PNL,profit
01 Jan 2013-31 Dec 2013,-95400.0000,806919.9998,806920.0000
01 Feb 2013-31 Jan 2014,-4500.0000,860200.0000,860200.0000
01 Mar 2013-28 Feb 2014,0.0000,864604.5451,875721.1968
01 Apr 2013-31 Mar 2014,-12167.9955,972527.6368,953005.1561
01 May 2013-30 Apr 2014,-51240.0000,817527.9607,821552.3243
01 Jun 2013-31 May 2014,0.0000,641440.0000,641630.0000
01 Jul 2013-30 Jun 2014,-84360.0000,830085.1470,830085.1470
01 Aug 2013-31 Jul 2014,-64800.0000,510342.4000,560934.4000
01 Sep 2013-31 Aug 2014,-44200.0000,546039.9999,543660.0000
01 Oct 2013-30 Sep 2014,-20440.9850,690131.0294,760969.0860


In [25]:
df = pd.DataFrame(optimization_df)

# Write the DataFrame to a CSV file
df.to_csv('output_train_12_1_group2_.csv', index=True)

In [26]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2014-31 Jan 2014,-51120.0000,-23080.0000,-42176.0000
01 Feb 2014-28 Feb 2014,-9000.0000,117000.0000,115100.0000
01 Mar 2014-31 Mar 2014,-45900.0000,55080.0000,53978.4000
01 Apr 2014-30 Apr 2014,-77759.9730,-28079.9865,-29452.7859
01 May 2014-31 May 2014,-29400.0000,-933.3333,-6888.0000
01 Jun 2014-30 Jun 2014,-8550.0000,31635.0000,17537.0000
01 Jul 2014-31 Jul 2014,-136800.0000,-119700.0000,-128250.0000
01 Aug 2014-31 Aug 2014,-115315.2000,20160.0000,-115315.2000
01 Sep 2014-30 Sep 2014,-4760.0000,163200.0000,166770.0000
01 Oct 2014-31 Oct 2014,-97549.6525,-50832.0442,-50767.3362


In [27]:
df = pd.DataFrame(testing_df)

# Write the DataFrame to a CSV file
df.to_csv('output_test_12_1_group2_.csv', index=True)

In [28]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2017','jan 2018',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=12,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jan 2016 31 Dec 2016
Data gathered for testing period: 01 Jan 2017 31 Jan 2017
optimisation completed
Optimal parameters are: [0.009, 1900000, 1.7, 20]
Optimization phase; Max loss, R_PNL, profit are: -147947.9999 827574.4313 830902.2302
Testing phase; Max loss, R_PNL, profit are: -15219.0 41193.85 30472.2
Data gathered for training period: 01 Feb 2016 31 Jan 2017
Data gathered for testing period: 01 Feb 2017 28 Feb 2017
optimisation completed
Optimal parameters are: [0.009, 800000, 1.4, 9]
Optimization phase; Max loss, R_PNL, profit are: -79200.0 978936.0291 961259.2584
Testing phase; Max loss, R_PNL, profit are: -43200.0 -5880.0 -13536.0
Data gathered for training period: 01 Mar 2016 28 Feb 2017
Data gathered for testing period: 01 Mar 2017 31 Mar 2017
optimisation completed
Optimal param

In [29]:
optimization_df

,max_loss,R_PNL,profit
01 Jan 2016-31 Dec 2016,-147947.9999,8.275744e+05,8.309022e+05
01 Feb 2016-31 Jan 2017,-79200.0000,9.789360e+05,9.612593e+05
01 Mar 2016-28 Feb 2017,0.0000,1.117997e+06,1.117997e+06
01 Apr 2016-31 Mar 2017,0.0000,1.083809e+06,1.083809e+06
01 May 2016-30 Apr 2017,-84510.0000,1.068296e+06,1.074500e+06
01 Jun 2016-31 May 2017,-89180.0000,8.843302e+05,8.921010e+05
01 Jul 2016-30 Jun 2017,0.0000,6.499721e+05,1.204184e+06
01 Aug 2016-31 Jul 2017,-7530.0000,8.521326e+05,1.301547e+06
01 Sep 2016-31 Aug 2017,-79275.3000,6.827031e+05,1.338403e+06
01 Oct 2016-30 Sep 2017,-174275.4000,1.215788e+06,1.254359e+06


In [30]:
df = pd.DataFrame(optimization_df)

# Write the DataFrame to a CSV file
df.to_csv('output_train_12_1_group2_2017.csv', index=True)

In [31]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2017-31 Jan 2017,-15219.000,41193.8500,30472.2000
01 Feb 2017-28 Feb 2017,-43200.000,-5880.0000,-13536.0000
01 Mar 2017-31 Mar 2017,-180784.000,-126000.0000,-180784.0000
01 Apr 2017-30 Apr 2017,-114502.245,-45745.9373,-54522.8568
01 May 2017-31 May 2017,-101817.000,-13822.2402,-23222.0000
01 Jun 2017-30 Jun 2017,0.000,64095.5463,35611.4880
01 Jul 2017-31 Jul 2017,-54381.600,7020.0000,-54381.6000
01 Aug 2017-31 Aug 2017,-36003.750,4289.7000,4034.5770
01 Sep 2017-30 Sep 2017,-35552.400,21373.4727,15397.0000
01 Oct 2017-31 Oct 2017,-9709.700,21462.1633,18494.0000


In [32]:
df = pd.DataFrame(testing_df)

# Write the DataFrame to a CSV file
df.to_csv('output_test_12_1_group2_2017.csv', index=True)

In [36]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2019','jan 2020',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=12,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period: 01 Jan 2018 31 Dec 2018
Data gathered for testing period: 01 Jan 2019 31 Jan 2019
optimisation completed
Optimal parameters are: [0.0055, 1800000, 1.7, 50]
Optimization phase; Max loss, R_PNL, profit are: -41184.0 1022625.4501 1009553.022
Testing phase; Max loss, R_PNL, profit are: -36432.0 53748.0 53748.0
Data gathered for training period: 01 Feb 2018 31 Jan 2019
Data gathered for testing period: 01 Feb 2019 28 Feb 2019
optimisation completed
Optimal parameters are: [0.0055, 1800000, 1.7, 50]
Optimization phase; Max loss, R_PNL, profit are: -38659.5 952138.1669 941414.4
Testing phase; Max loss, R_PNL, profit are: -59400.0 7101.0 7101.0
Data gathered for training period: 01 Mar 2018 28 Feb 2019
Data gathered for testing period: 01 Mar 2019 31 Mar 2019
optimisation completed
Optimal parameters 

In [39]:
optimization_df

,max_loss,R_PNL,profit
01 Jan 2018-31 Dec 2018,-41184.000,1.022625e+06,1.009553e+06
01 Feb 2018-31 Jan 2019,-38659.500,9.521382e+05,9.414144e+05
01 Mar 2018-28 Feb 2019,-33516.000,8.981070e+05,8.847270e+05
01 Apr 2018-31 Mar 2019,-9600.000,1.013542e+06,1.013542e+06
01 May 2018-30 Apr 2019,-78400.000,9.660000e+05,9.660000e+05
01 Jun 2018-31 May 2019,-34272.000,7.973663e+05,8.172411e+05
01 Jul 2018-30 Jun 2019,-21600.000,7.896000e+05,7.969600e+05
01 Aug 2018-31 Jul 2019,-56000.000,7.364741e+05,7.350324e+05
01 Sep 2018-31 Aug 2019,-16.800,7.834671e+05,7.834671e+05
01 Oct 2018-30 Sep 2019,-40412.568,8.859881e+05,8.643264e+05


In [ ]:
df = pd.DataFrame(optimization_df)

# Write the DataFrame to a CSV file
df.to_csv('output_train_12_1_group2_2019.csv', index=True)

In [37]:
testing_df

,max_loss,R_PNL,profit
01 Jan 2019-31 Jan 2019,-36432.0000,53748.0000,53748.0000
01 Feb 2019-28 Feb 2019,-59400.0000,7101.0000,7101.0000
01 Mar 2019-31 Mar 2019,-31500.0000,45255.0000,29655.0000
01 Apr 2019-30 Apr 2019,-18816.0000,13898.5075,2624.0000
01 May 2019-31 May 2019,-14000.0000,14000.0000,16800.0000
01 Jun 2019-30 Jun 2019,-37418.4000,9360.0000,-37418.4000
01 Jul 2019-31 Jul 2019,-33720.0000,19200.0000,-33720.0000
01 Aug 2019-31 Aug 2019,0.0000,46252.2958,26540.4200
01 Sep 2019-30 Sep 2019,0.0000,71997.6678,66072.6320
01 Oct 2019-31 Oct 2019,-134156.8200,-69018.5480,-99317.4560


In [38]:
df = pd.DataFrame(testing_df)

# Write the DataFrame to a CSV file
df.to_csv('output_test_12_1_group2_2019.csv', index=True)

In [6]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)

n_combination = n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params

print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_multiplier_params*n_lookback_params))


# Adjust these parameter according to search space
n_trials = 50 #NGEN
npop = 100
early_stopping_gen = n_trials # no early stopping
optimizer_param = [n_trials, npop, early_stopping_gen]

parameters = [grid_params,lot_params,multiplier_params,lookback_params]

testing_df, optimization_df = walk_forward_analysis_multiplier('jan 2016','jan 2017',1,parameters,optimization_function=deap_optimiser_multiplier,optimizer_params=optimizer_param,lookback_in_months=12,evaluation_period=1)


number of grid params:- 18.0
number of lot params:- 19.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 47880.0
Data gathered for training period:  01 Jan 2015 31 Dec 2015
Data gathered for testing period:  01 Jan 2016 31 Jan 2016
optimisation completed
Optimal parameters are:  [0.0015, 200000, 2.9, 30]
Optimization phrase; Max loss,R_PNL,profit are:  -58490.7 1316896.9764 1317317.8759
Testing phrase; Max loss, R_PNL,profit are:  -219665.52 -181518.568 -120107.808
Data gathered for training period:  01 Feb 2015 31 Jan 2016
Data gathered for testing period:  01 Feb 2016 29 Feb 2016
optimisation completed
Optimal parameters are:  [0.0075, 1700000, 0.9, 50]
Optimization phrase; Max loss,R_PNL,profit are:  -128269.5975 1235861.0734 1230966.24
Testing phrase; Max loss, R_PNL,profit are:  -229258.9038 -118193.6538 -229258.9038
Data gathered for training period:  01 Mar 2015 29 Feb 2016
Data gathered for testing period:  01 Mar 2016 31 Mar 2016

ValueError: too many values to unpack (expected 2)

In [7]:
testing_df

NameError: name 'testing_df' is not defined